In [ ]:
import warnings
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import gensim as gs
from tqdm import tqdm
import tensorflow as tf
import matplotlib.pyplot as plt
from nltk.tokenize import TweetTokenizer
tknzr = TweetTokenizer()
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
warnings.filterwarnings('ignore')
import fasttext

In [ ]:
train_dataset = pd.read_csv('train_data_1M.csv', delimiter=',')
validation_dataset = pd.read_csv('validation_data_1M.csv', delimiter=',')

In [ ]:
embedding_model = gs.models.FastText.load('../train_embedding_models/fasttext_embedding_50d_all_signals')

In [ ]:
MAX_PAD = 31

def sequence_padding(X_DIM, value):
    
    value_padding = np.pad(value, ((0,MAX_PAD - X_DIM),(0,0)), 'constant')
    
    return value_padding

In [ ]:
def create_embedding(value):

    value = tknzr.tokenize(str(value))
    
    if len(value) < MAX_PAD:
        
        embedding = embedding_model.wv[value]
        embedding = embedding.astype('float16')
        
        padding_embedding = sequence_padding(embedding.shape[0],embedding)
        
        return padding_embedding
        
    else:
        
        embedding = embedding_model.wv[value[0:MAX_PAD]]
        embedding = embedding.astype('float16')
        
        return embedding

In [ ]:
train_article_title = []
train_table_title = []
train_label = []

for i,row in tqdm(train_dataset.iterrows()):
    
    train_article_title.append(create_embedding(row['article_page_title']))
    train_table_title.append(create_embedding(row['table_page_title']))
    train_label.append(row['label'])

In [ ]:
train_article_title = np.array(train_article_title,dtype='float16')
train_table_title = np.array(train_table_title,dtype='float16')
train_label = np.array(train_label)

In [ ]:
validation_article_title = []
validation_table_title = []
validation_label = []

for i, row in tqdm(validation_dataset.iterrows()):
    
    validation_article_title.append(create_embedding(row['article_page_title']))
    validation_table_title.append(create_embedding(row['table_page_title']))  
    validation_label.append(row['label'])

In [ ]:
validation_article_title = np.array(validation_article_title,dtype='float16')
validation_table_title = np.array(validation_table_title,dtype='float16')
validation_label = np.array(validation_label)

In [ ]:
def attention_method(query, key, value):
    
    scores = tf.matmul(query, key, transpose_b=True)
    
    distribution = tf.nn.softmax(scores)
    
    attention_matrix = tf.matmul(distribution, value)
    
    return attention_matrix

In [ ]:
def attention_model():
    
    article_title = tf.keras.Input(shape=(31,50), dtype='float32')
    table_title = tf.keras.Input(shape=(31,50), dtype='float32')
    
    context_titleA = tf.keras.layers.Bidirectional(tf.keras.layers.GRU(32, return_sequences=True))(article_title)
    context_titleB = tf.keras.layers.Bidirectional(tf.keras.layers.GRU(32, return_sequences=True))(table_title)
    
    affinity_titleA_titleB = attention_method(context_titleA,context_titleB,context_titleA)
    
    context_affinity = tf.keras.layers.Bidirectional(tf.keras.layers.GRU(32, return_sequences=True))(affinity_titleA_titleB)
    
    learned_vector = tf.keras.layers.Flatten()(context_affinity)
   
    MLP_input = tf.keras.layers.Dense(64, activation='relu')(learned_vector)
    dropout1 = tf.keras.layers.Dropout(0.5, name="dropout1")(MLP_input)
    MLP_hidden = tf.keras.layers.Dense(32, activation='relu')(dropout1)
    dropout2 = tf.keras.layers.Dropout(0.5, name="dropout2")(MLP_hidden)
    MLP_output = tf.keras.layers.Dense(1,activation='sigmoid')(dropout2)
    
    model = tf.keras.Model(inputs=[article_title,table_title],outputs=MLP_output)
    
    return model

In [ ]:
model = attention_model()

In [ ]:
model.compile(loss="binary_crossentropy",optimizer="adam", metrics=['accuracy'])

In [ ]:
filepath="best_model_title_attention_{epoch:02d}_{val_accuracy:.4f}.h5"
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath, monitor='val_accuracy', verbose=2, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

In [ ]:
history = model.fit([train_article_title, train_table_title], train_label, 
          epochs=30, 
          batch_size=32,
          verbose=2,
          validation_data=([validation_article_title, validation_table_title], validation_label),
          callbacks=callbacks_list)